# pandapower Optimal Power Flow
This is an introduction into the usage of the pandapower optimal power flow. It shows how to set the constraints and the cost factors into the pandapower element tables.

## Example Network

We use the following four bus example network for this tutorial:

<img src="pics/example_opf.png" width="50%">

We first create this network in pandapower:

In [46]:
import pandapower as pp
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=220.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)

#create 220/110 kV transformer
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")

#create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

#create loads
pp.create_load(net, bus2, p_kw=60e3)
pp.create_load(net, bus3, p_kw=70e3)
pp.create_load(net, bus4, p_kw=10e3)

#create generators
eg = pp.create_ext_grid(net, bus1)
g0 = pp.create_gen(net, bus3, p_kw=-80*1e3, min_p_kw=0, max_p_kw=-80e3,vm_pu=1.01, controllable=True)
g1 = pp.create_gen(net, bus4, p_kw=-100*1e3, min_p_kw=0, max_p_kw=-100e3, vm_pu=1.01, controllable=True)

## Loss Minimization

We run an OPF:

In [47]:
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


let's check the results:

In [48]:
net.res_ext_grid

,p_kw,q_kvar
0,-56530.152093,-1974.474112


In [49]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-71313.868104,1969.65441,-3.712792,1.00001
1,-12299.269570,1451.16052,-3.712792,1.00001


Since no individiual generation costs were specified, the OPF minimizes overall power generation, which is equal to a loss minimization in the network. The loads at buses 3 and 4 are supplied by generators at the same bus, the load at Bus 2 is provided by a combination of the other generators so that the power transmission leads to minimal losses.

## Individual Generator Costs

Let's now assign individual costs to each generator.

We assign a cost of 10 ct/kW for the external grid, 15 ct/kw for the generator g0 and 12 ct/kw for generator g1:

In [50]:
net.ext_grid.loc[eg, "cost_per_kw"] = 0.10
net.gen.loc[g0, "cost_per_kw"] = 0.15
net.gen.loc[g1, "cost_per_kw"] = 0.12

And now run an OPF:

In [51]:
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


We can see that all active power is provided by the external grid. This makes sense, because the external grid has the lowest cost of all generators and we did not define any constraints.

We define a simple function that calculates the summed costs:

In [52]:
def calc_costs(net):
    cost_gen = (-net.res_gen.p_kw * net.gen.cost_per_kw).sum()
    cost_eg = (-net.res_ext_grid.p_kw * net.ext_grid.cost_per_kw).sum()
    return (cost_gen + cost_eg) * 1e-3

And calculate the dispatch costs:

In [53]:
calc_costs(net)

14.455949422013624

### Transformer Constraint

Since all active power comes from the external grid and subsequently flows through the transformer, the transformer is overloaded with a loading of about 145%:

In [54]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,144559.494205,9193.074478,-143959.565291,15993.751648,599.928914,25186.826126,0.380137,0.759989,144.85151


We now limit the transformer loading to 50%:

In [55]:
net.trafo["max_loading_percent"] = 50

(the max_loading_percent parameter can also be specified directly when creating the transformer)
and run the OPF:

In [56]:
pp.runopp(net)

We can see that the transformer complies with the maximum loading:

In [57]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,49999.873587,112.43221,-49879.973547,2907.574865,119.90004,3020.007076,0.131216,0.262147,50.0


And power generation is now split between the external grid and generator 1 (which is the second cheapest generation unit):

In [58]:
net.res_ext_grid

,p_kw,q_kvar
0,-49999.873587,-112.43221


In [59]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-79999.999995,1537.641177,-2.700424,1.007686
1,-10183.797563,786.692337,-3.114347,1.004213


This comes of course with an increase in dispatch costs:

In [42]:
calc_costs(net)

18.222043065621488

### Line Loading Constraints

Wen now look at the line loadings:

In [43]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,-5634.036570,-1546.075409,5669.797299,-725.508565,35.760730,-2271.583974,0.030653,0.029773,0.030653,6.521832
1,4330.202696,-812.132612,-4315.398796,-839.515642,14.803901,-1651.648254,0.022948,0.022978,0.022978,4.888888
2,4499.196359,52.823305,-4485.989883,-1361.499456,13.206476,-1308.676151,0.023517,0.024597,0.024597,5.233306


and run the OPF with a 50% loading constraint:

In [44]:
net.line["max_loading_percent"] = 50
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


Now the line loading constraint is complied with:

In [45]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,-5634.036570,-1546.075409,5669.797299,-725.508565,35.760730,-2271.583974,0.030653,0.029773,0.030653,6.521832
1,4330.202697,-812.132612,-4315.398796,-839.515642,14.803901,-1651.648254,0.022948,0.022978,0.022978,4.888888
2,4499.196358,52.823305,-4485.989882,-1361.499456,13.206476,-1308.676151,0.023517,0.024597,0.024597,5.233306


And all generators are involved in supplying the loads:

In [19]:
net.res_ext_grid

,p_kw,q_kvar
0,-49787.580003,4603.457521


In [20]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-9136.450967,-2430.689358,-5.815435,0.993014
1,-83592.613615,-4853.769506,-1.511329,1.028887


This of course comes with a once again rising dispatch cost:

In [21]:
calc_costs(net)

16.38033927907391

### Voltage Constraints

Finally, we have a look at the bus voltage:

In [22]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-49787.580003,4603.457521
1,1.006024,-3.408832,60000.000000,0.000000
2,0.993014,-5.815435,60863.549033,-2430.689358
3,1.028887,-1.511329,-73592.613615,-4853.769506


and constrain it:

In [23]:
net.bus["min_vm_pu"] = 1.0
net.bus["max_vm_pu"] = 1.02
pp.runopp(net)

We can see that all voltages are within the voltage band:

In [24]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-49906.817982,3050.590592
1,1.004168,-3.421013,60000.000000,0.000000
2,1.000000,-5.976090,59278.074279,-14859.213746
3,1.020000,-1.366887,-71863.385929,9173.011326


And all generators are once again involved in supplying the loads:

In [25]:
net.res_ext_grid

,p_kw,q_kvar
0,-49906.817982,3050.590592


In [26]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-10721.925721,-14859.213746,-5.976090,1.00
1,-81863.385929,9173.011326,-1.366887,1.02


This of course comes once again with rising dispatch costs:

In [27]:
calc_costs(net)

16.422576967990047

## Minimizing Active Power Curtailment

Now we assume that the generators are renewable energy sources, and we want to feed in as much of the energy they can provide as possible without violating any constraints.

We assign negative costs to the generators and costs of zero for the external grid.

In [28]:
net.ext_grid.cost_per_kw = 0
net.gen.cost_per_kw = -1
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


Because of the negative costs, the OPF now maximizes power generation at the generators:

In [31]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-46052.644816,-18912.998087,-0.485062,1.02
1,-85097.782114,19478.228436,2.869966,1.02


In [32]:
net.res_ext_grid

,p_kw,q_kvar
0,-11071.74798,153.958385


In [33]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.00,0.000000,-11071.747980,153.958385
1,1.00,-0.759444,60000.000000,0.000000
2,1.02,-0.485062,23947.355184,-18912.998087
3,1.02,2.869966,-75097.782114,19478.228436


In [34]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,11071.74798,-153.958385,-11013.579078,324.308603,58.168902,170.350217,0.029059,0.057831,11.072818


In [35]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_ka,loading_percent
0,-4940.672063,-7253.499354,5009.683533,5024.003563,69.011470,-2229.495791,0.046064,9.800775
1,-28957.038717,13888.994524,29770.861583,-13899.328668,813.822866,-10.334143,0.169066,35.971544
2,45326.920531,-5578.899768,-44045.748858,6929.190751,1281.171673,1350.290984,0.235000,49.999973


Obviously the voltage profile was the limiting factor for the generator feed-in. If we relax this constraint a little bit:

In [36]:
net.bus.max_vm_pu = 1.05
pp.runopp(net)

We see an increased feed-in of the generators:

In [37]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-79999.991970,-17252.186816,2.328271,1.050000
1,-66517.876366,10225.945538,3.665995,1.040019


In [38]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,4542.414746,8523.970418
1,1.010373,0.298333,60000.000000,0.000000
2,1.050000,2.328271,-9999.991970,-17252.186816
3,1.040019,3.665995,-56517.876366,10225.945538
